# Introduction
This notebook contains code for calculating and visualizing various vegetation indices from a satellite image using `rasterio`. The indices calculated include NDVI, NDMI, NBR, NDWI, BAI, BAIML, BAIMS, MiRBi, and GEMI. The calculated indices are then saved as separate GeoTIFF files. Additionally, some of the indices are visualized using `matplotlib` and saved as JPEG images.

## Importing Libraries
First, we import the required libraries: `rasterio`, `numpy`, `os`, and `matplotlib.pyplot`.

In [1]:

import rasterio as rio
import numpy as np
import os
import matplotlib.pyplot as plt
from rasterio.plot import show, show_hist


### Loading Bands
We load the bands from the input image file and store them in a dictionary called bands_dict. Each band is stored as a NumPy array.
- Input file is a image stack that created in ENVI
- The input mask is also created in ENVI
- I will soon update this with a full workflow without ENVI.

In [ ]:
#set directory
os.chdir('/Users/najah/work/internships/meghna/LT05_L1TP_145044_20100428_20161016_01_T1')
image_file = './145044_masked_stacked_wgs84.tif'
bands_dict = {}

with rio.open(image_file) as src:
    for bands in range(1, 7):
        bands_dict[str('b') + str(bands)] = src.read(bands)

# Renaming b6 to b7 since there's no thermal band
bands_dict['b7'] = bands_dict['b6']
del bands_dict['b6']


### Reading Mask File
To apply a mask to the generated indices, we'll read a mask file using rasterio and store it in the bands_dict dictionary.

In [ ]:
mask_file = rio.open('./145044_20100428_mask_all.tif')
mask = mask_file.read(1)
bands_dict['mask'] = mask



### Define the indices and their corresponding bands



In [ ]:

indices = {
    'ndvi': ('b4', 'b3'),
    'ndmi': ('b4', 'b5'),
    'nbr': ('b4', 'b7'),
    'ndwi': ('b2', 'b4'),
    'bai': ('b3', 'b4'),
    'baiml': ('b4', 'b7'),
    'baims': ('b4', 'b5'),
    'mirbi': ('b7', 'b5'),
    'nbrt': ('b4', 'b7', 'b6'),
    'gemi': ('b4', 'b3')
}



### Calculate the indices



In [ ]:

for index, bands in indices.items():
    if index == 'bai':
        bands_dict[index] = 1 / ((0.1 - bands_dict[bands[0]].astype(float)) ** 2 + (0.06 - bands_dict[bands[1]].astype(float)) ** 2)
    elif index == 'baiml':
        bands_dict[index] = 1 / ((bands_dict[bands[0]] - (.5 * bands_dict[bands[0]])) ** 2 + (bands_dict[bands[1]] - (.2 * bands_dict[bands[1]])) ** 2)
    elif index == 'baims':
        bands_dict[index] = 1 / ((bands_dict[bands[0]] - (.5 * bands_dict[bands[0]])) ** 2 + (bands_dict[bands[1]] - (.2 * bands_dict[bands[1]])) ** 2)
    elif index == 'mirbi':
        bands_dict[index] = (10 * bands_dict[bands[0]]) - ((9.8 * bands_dict[bands[1]]) + 2)
    elif index == 'nbrt':
        bands_dict[index] = (bands_dict[bands[0]] - bands_dict[bands[1]] * (bands_dict[bands[2]] / 1000)) / (bands_dict[bands[0]] + bands_dict[bands[1]] * (bands_dict[bands[2]] / 1000))
    elif index == 'gemi':
        eta = (2 * (bands_dict[bands[0]] ** 2 - bands_dict[bands[1]] ** 2) + (1.5 * bands_dict[bands[0]]) + (.5 * bands_dict[bands[1]])) / (bands_dict[bands[0]] + bands_dict[bands[1]] + .5)
        bands_dict[index] = eta * (1 - .25 * eta) - ((bands_dict[bands[1]] - .125) / (1 - bands_dict[bands[1]]))
    else:
        bands_dict[index] = (bands_dict[bands[0]].astype(float) - bands_dict[bands[1]].astype(float)) / (bands_dict[bands[0]] + bands_dict[bands[1]])

    bands_dict[index][bands_dict['mask'] == 0] = np.nan

    print(index + " created successfully")



### Save indices as separate GeoTIFF files

In [ ]:


output_dir = './indices1/'
os.makedirs(output_dir, exist_ok=True)

for index, array in bands_dict.items():
    if index != 'mask':
        output_file = os.path.join(output_dir, index + '.tif')
        with rio.open(output_file, 'w', **src.profile) as dst:
            dst.write(array, 1)



### Visualize and save some indices as JPEG images

In [ ]:


output_images_dir = './index_images/'
os.makedirs(output_images_dir, exist_ok=True)

for index in ['ndvi', 'ndmi', 'nbr']:
    plt.figure(figsize=(10, 10))
    plt.imshow(bands_dict[index], cmap='RdYlGn')
    plt.colorbar()
    plt.title(index.upper())
    plt.axis('off')
    output_image_file = os.path.join(output_images_dir, index + '.jpg')
    plt.savefig(output_image_file, bbox_inches='tight', dpi=300)
    plt.close()

print("All indices calculated and saved successfully!")